In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [3]:
now = datetime.now().date()
sdate= datetime.now().date()-timedelta(days=365*2)
datediff = (now-sdate).days

In [227]:
allwind = pd.read_pickle('brommer_raw_2020-10-04_2022-10-03.pkl')

In [228]:
allwind[::len(allwind)//10]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
0,2020-10-04,12:04 AM,Fall,Oct,Overnight,55.6,East,90.0,0.0,0.0
20405,2020-12-14,12:34 AM,Winter,Dec,Overnight,45.4,WSW,247.5,0.0,0.9
40810,2021-02-23,12:24 AM,Winter,Feb,Overnight,49.4,WSW,247.5,0.0,0.0
61215,2021-05-20,7:24 AM,Spring,May,Morning,47.6,NE,45.0,1.2,4.0
81620,2021-07-30,1:29 PM,Summer,Jul,Midday,63.9,SW,225.0,4.3,8.0
102025,2021-10-09,4:29 PM,Fall,Oct,Afternoon,67.8,WSW,247.5,3.0,8.0
122430,2021-12-19,9:52 PM,Winter,Dec,Evening,51.1,SSW,202.5,7.0,8.0
142835,2022-03-01,8:14 PM,Spring,Mar,Evening,54.9,East,90.0,0.9,6.0
163240,2022-05-12,12:24 AM,Spring,May,Overnight,47.7,NNE,22.5,2.4,4.0
183645,2022-07-23,12:19 AM,Summer,Jul,Overnight,56.2,West,270.0,0.9,3.0


In [284]:
def dftostrFF(df):
    print(df.reset_index(drop=False).to_string(index=False))
def dftostrTT(df):
    print(df.reset_index(drop=True).to_string(index=True))
def dftostrTF(df):
    print(df.reset_index(drop=True).to_string(index=False))
def dftostrFT(df):
    print(df.reset_index(drop=False).to_string(index=True))

In [226]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}
revdir = [(value, key) for key, value in ordir.items()]
print('mapping of 16 angular wind directions to 16 compass points:\n')
print(pd.DataFrame(revdir,columns=['Azimuth','Compass']).to_string(index=False))
revord = dict(revdir)

mapping of 16 angular wind directions to 16 compass points:

 Azimuth Compass
     0.0   North
    22.5     NNE
    45.0      NE
    67.5     ENE
    90.0    East
   112.5     ESE
   135.0      SE
   157.5     SSE
   180.0   South
   202.5     SSW
   225.0      SW
   247.5     WSW
   270.0    West
   292.5     WNW
   315.0      NW
   337.5     NNW


In [352]:
def ordinalify(df,groupcol,grouporder):
    avgdir = df.groupby(groupcol)['Azimuth'].mean()[grouporder]
    avgint = avgdir.astype(int)
    clsdir = (avgdir//22.5)*22.5
    dford = pd.concat([pd.concat([pd.DataFrame(avgint),pd.DataFrame(clsdir)],axis=1),clsdir.map(revord)],axis=1)
    dford.columns=['Azimuth','Closest','Compass']
    dford = dford.reset_index(drop=False)
    return dford

In [291]:
print("average wind speed by season and daypart:\n")
dfs = allwind.groupby(['Season','Daypart'])['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Season'] = pd.Categorical(dfs['Season'], ['Winter','Spring','Summer','Fall'])
dfs['Daypart'] = pd.Categorical(dfs['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dftostrTF(dfs.sort_values(['Season','Daypart']))

average wind speed by season and daypart:

Season   Daypart  Speed
Winter Overnight    1.3
Winter   Morning    1.3
Winter    Midday    3.8
Winter Afternoon    2.8
Winter   Evening    1.5
Spring Overnight    1.2
Spring   Morning    1.6
Spring    Midday    4.6
Spring Afternoon    5.1
Spring   Evening    3.0
Summer Overnight    1.2
Summer   Morning    1.7
Summer    Midday    4.1
Summer Afternoon    4.9
Summer   Evening    3.2
  Fall Overnight    0.9
  Fall   Morning    1.2
  Fall    Midday    3.1
  Fall Afternoon    2.9
  Fall   Evening    1.5


In [293]:
print("ascending avg wind speed by season and daypart:\n")
dftostrTF(dfs.sort_values('Speed'))

ascending avg wind speed by season and daypart:

Season   Daypart  Speed
  Fall Overnight    0.9
Spring Overnight    1.2
Summer Overnight    1.2
  Fall   Morning    1.2
Winter   Morning    1.3
Winter Overnight    1.3
  Fall   Evening    1.5
Winter   Evening    1.5
Spring   Morning    1.6
Summer   Morning    1.7
Winter Afternoon    2.8
  Fall Afternoon    2.9
Spring   Evening    3.0
  Fall    Midday    3.1
Summer   Evening    3.2
Winter    Midday    3.8
Summer    Midday    4.1
Spring    Midday    4.6
Summer Afternoon    4.9
Spring Afternoon    5.1


In [355]:
print("unweighted average wind azimuth direction by season:\n")
orderszn = ['Winter','Spring','Summer','Fall']
dftostrTF(ordinalify(allwind,'Season',orderszn))
print("\n\nunweighted average wind azimuth direction by month:\n")
ordermth=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
dftostrTF(ordinalify(allwind,'Month',ordermth))

unweighted average wind azimuth direction by season:

Season  Azimuth  Closest Compass
Winter      175    157.5     SSE
Spring      144    135.0      SE
Summer      159    157.5     SSE
  Fall      157    135.0      SE


unweighted average wind azimuth direction by month:

Month  Azimuth  Closest Compass
  Jan      164    157.5     SSE
  Feb      190    180.0   South
  Mar      167    157.5     SSE
  Apr      107     90.0    East
  May      158    157.5     SSE
  Jun      161    157.5     SSE
  Jul      158    157.5     SSE
  Aug      157    135.0      SE
  Sep      156    135.0      SE
  Oct      155    135.0      SE
  Nov      160    157.5     SSE
  Dec      173    157.5     SSE


In [340]:
def groupwavg(groupcol,grouporder):
    wgtdir = (nnwind['Speed']).multiply(nnwind['Orient'])
    dfwgt = pd.concat([nnwind,pd.DataFrame(wgtdir,columns=['Wgtdir'])],axis=1)[[groupcol,'Speed','Wgtdir']]
    dfsum = dfwgt.groupby(groupcol).sum()
    wavgd = (dfsum.Wgtdir/dfsum.Speed)
    wintd = wavgd.astype(int)
    wclsd = (wavgd//22.5)*22.5
    wcomp = wclsd.map(revord)
    dfwavg = pd.concat([pd.concat([wintd,wclsd],axis=1),wcomp],axis=1)
    dfwavg.columns = ['Azimuth','Closest','Compass']
    dfwavg = dfwavg.reset_index(drop=False)
    dfwavg[groupcol] = pd.Categorical(dfwavg[groupcol], grouporder)
    return dfwavg.sort_values(groupcol)

In [341]:
print("speed weighted average wind azimuth direction by Season:\n")
dftostrTF(groupwavg('Season',['Winter','Spring','Summer','Fall']))
print("\n\nspeed weighted average wind azimuth direction by Month:\n")
dftostrTF(groupwavg('Month',ordermth))

speed weighted average wind azimuth direction by Season:

Season  Azimuth  Closest Compass
Winter      185    180.0   South
Spring      165    157.5     SSE
Summer      161    157.5     SSE
  Fall      169    157.5     SSE


speed weighted average wind azimuth direction by Month:

Month  Azimuth  Closest Compass
  Jan      174    157.5     SSE
  Feb      204    202.5     SSW
  Mar      185    180.0   South
  Apr      132    112.5     ESE
  May      177    157.5     SSE
  Jun      166    157.5     SSE
  Jul      159    157.5     SSE
  Aug      158    157.5     SSE
  Sep      161    157.5     SSE
  Oct      170    157.5     SSE
  Nov      180    180.0   South
  Dec      175    157.5     SSE


In [389]:
print("difference in monthly average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(ordermth),(ordinalify(allwind,'Month',ordermth)).Azimuth.subtract((groupwavg('Month',ordermth)).Azimuth)],axis=1)
dfdiff.columns=['Month','Azimuth']
dftostrTF(dfdiff)
print("\n\ndifference in seasonal average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(orderszn),(ordinalify(allwind,'Season',orderszn)).Azimuth.subtract((groupwavg('Season',orderszn)).Azimuth)],axis=1)
dfdiff.columns=['Season','Azimuth']
dftostrTF(dfdiff)

difference in monthly average azimuth without vs with wind speed as weights:

Month  Azimuth
  Jan       32
  Feb       32
  Mar       -8
  Apr      -97
  May      -16
  Jun        2
  Jul       -8
  Aug      -28
  Sep      -21
  Oct      -25
  Nov      -10
  Dec       12


difference in seasonal average azimuth without vs with wind speed as weights:

Season  Azimuth
Winter        6
Spring      -21
Summer       -2
  Fall      -28
